## INITIALIZE


In [7]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf
from model_set import Custom1DCNN
import capilab_dataset2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

label_keys = {'ball':1,'box':2,'pen':3,'no':0}

X, y = capilab_dataset2.get(['Datasets\Lai_JulyData.mat', 'Datasets\Lai_JulyData.mat', 'Datasets\Suguro_JulyData.mat', 'Datasets\Takahashi_JulyData.mat']) 

try:
    _x, x_test,  _y, y_test = train_test_split(X, y, test_size = .1, stratify = y, random_state = 420)
    x_train, x_val, y_train, y_val = train_test_split(_x, _y, test_size = .2, stratify = _y, random_state = 420)
except Exception as e:
    print(e)



ckpt = ModelCheckpoint('tkhDatasetTest/',monitor='val_loss', verbose=False,
                                    save_best_only=True, save_weights_only=False, mode='auto',save_freq="epoch")

es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, restore_best_weights=True, verbose = False)
cb_list = [ckpt, es]


c:\Users\capiAdmin\anaconda3\envs\dqn_rlsel\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [3]:
#STFU
from silence_tensorflow import silence_tensorflow
silence_tensorflow()

### EXP I
- Dataset:********* capilab_dataset2
- Classifier:****** Custom1DCNN
- Preprocessor:**** None
- Channel:********* ALL


In [46]:
_x_train = x_train.copy()
_x_test = x_test.copy()
_x_val = x_val.copy()

vl = []
va = []


with tqdm(total = 10, position = 0, leave = True) as pbar:
    for e in tqdm(range(10), ncols = 100, position = 0, leave = True, desc ="getting the result>"):

        opt = tf.keras.optimizers.Adam(learning_rate = 1e-4)
        loss = tf.keras.losses.CategoricalCrossentropy()
        model = Custom1DCNN(inp_shape=(_x_train.shape[1], _x_train.shape[2]), output_classes=y_train.shape[1])
        model.compile(optimizer = opt, loss=loss , metrics=['accuracy'])

        hist = model.fit(_x_train, y_train, batch_size=16, epochs = 70, verbose = False, validation_data = (_x_val,y_val), callbacks = cb_list)

        acc = hist.history['accuracy']
        val_acc = hist.history['val_accuracy']
        loss = hist.history['loss']
        val_loss = hist.history['val_loss']
        #eval
        vloss, vacc =  model.evaluate(_x_val, y_val, verbose = False)
        va.append(vacc)
        vl.append(vloss)

print("Mean L|ACC and STD L|ACC = {}|{} {}|{} ".format(
                                                        np.mean(np.array(vl)),
                                                        np.mean(np.array(va)),
                                                        np.std(np.array(vl)),
                                                        np.std(np.array(va))
                                                    )
    )




# yPred = model.predict(x_test)
# yTestClass = np.argmax(y_test, axis=1)
# yPredClass = np.argmax(yPred,axis=1)

# report =  classification_report(yTestClass, yPredClass, target_names=["No Move", "Ball", 'Box / Mobile', 'Pen'])
# cf_matrix = confusion_matrix(yTestClass,yPredClass)
# print('\n----------<Classification report>----------\n\n',report)

# sns.set(rc={'figure.figsize':(10,10)})
# sns.set(font_scale=2)
# xticks = yticks = ['NM', 'BL', 'PX', 'PN']
# sns.heatmap(cf_matrix, annot=True, xticklabels = xticks, yticklabels=yticks, cmap='gray', linewidth=.5)

# plt.figure(figsize=(20,7))
# sns.set(font_scale=1)
# plt.subplot(121), plt.plot(acc, label='acc'),plt.plot(val_acc,label='val_acc'),plt.title("Accuracy VS Validation Accuracy")
# plt.legend()
# plt.subplot(122), plt.plot(loss, label='loss'),plt.plot(val_loss,label='val_loss'),plt.title("Loss VS Validation Loss")
# plt.legend()
# plt.suptitle('Experiment 1', fontsize = 20)
# plt.show()


  0%|          | 0/10 [16:31<?, ?it/s]

Mean L|ACC and STD L|ACC = 0.8619410812854766|0.6798611164093018 0.038156608795933335|0.01972488408407706 


### EXP II
- Dataset:********* capilab_dataset2
- Classifier:****** Custom1DCNN
- Preprocessor:**** None
- Channel:********* REMOVE -> ['C4','F4', 'Fp2','Fp1','F3', 'O1', 'F7', 'F8', 'Fz']


In [40]:
dataset_channel_map = {0: 'F4', 1: 'C4', 2: 'Pa', 3: 'Cz', 4: 'F3', 5: 'C3', 6: 'P3', 7: 'F7', 8: 'T3', 
                                    9: 'T5', 10: 'Fp1', 11: 'Fp2', 12: 'T4', 13: 'F8', 14: 'Fz', 15: 'Pz', 16: 'T6', 17: 'O2', 18: 'O1'}

removed_ch = ['C4','F4', 'Fp2','Fp1','F3', 'O1', 'F7', 'F8', 'Fz']
target = [k for k,v in dataset_channel_map.items() if v not in removed_ch]

_x_train = x_train[:,:,target].copy()
_x_test = x_test[:,:,target].copy()
_x_val = x_val[:,:,target].copy()

vl = []
va = []

with tqdm(total = 10, position = 0, leave = True) as pbar:
    for e in tqdm(range(10), ncols = 100, position = 0, leave = True, desc ="getting the result>"):

        opt = tf.keras.optimizers.Adam(learning_rate = 1e-4)
        loss = tf.keras.losses.CategoricalCrossentropy()
        model = Custom1DCNN(inp_shape=(_x_train.shape[1], _x_train.shape[2]), output_classes=y_train.shape[1])
        model.compile(optimizer = opt, loss=loss , metrics=['accuracy'])

        hist = model.fit(_x_train, y_train, batch_size=16, epochs = 70, verbose = False, validation_data = (_x_val,y_val), callbacks = cb_list)

        acc = hist.history['accuracy']
        val_acc = hist.history['val_accuracy']
        loss = hist.history['loss']
        val_loss = hist.history['val_loss']
        #eval
        vloss, vacc =  model.evaluate(_x_val, y_val, verbose = False)
        va.append(vacc)
        vl.append(vloss)

print("Mean L|ACC and STD L|ACC = {}|{} {}|{} ".format(
                                                        np.mean(np.array(vl)),
                                                        np.mean(np.array(va)),
                                                        np.std(np.array(vl)),
                                                        np.std(np.array(va))
                                                    )
    )
    

  0%|          | 0/10 [14:10<?, ?it/s]

Mean L|ACC and STD L|ACC = 0.9791095137596131|0.5958333283662796 0.13865926792012134|0.1125685664355553 


### EXP III
- Dataset:********* capilab_dataset2
- Classifier:****** Custom1DCNN
- Preprocessor:**** None
- Channel:********* BEST_CH = (C3, C4)


In [38]:
from tqdm import tqdm
vl = []
va = []

dataset_channel_map = {0: 'F4', 1: 'C4', 2: 'Pa', 3: 'Cz', 4: 'F3', 5: 'C3', 6: 'P3', 7: 'F7', 8: 'T3', 
                                    9: 'T5', 10: 'Fp1', 11: 'Fp2', 12: 'T4', 13: 'F8', 14: 'Fz', 15: 'Pz', 16: 'T6', 17: 'O2', 18: 'O1'}

target = [1, 5]
_x_train = x_train[:,:,target].copy()
_x_test = x_test[:,:,target].copy()
_x_val = x_val[:,:,target].copy()


with tqdm(total = 10, position = 0, leave = True) as pbar:
    for e in tqdm(range(10), ncols = 100, position = 0, leave = True, desc ="getting the result>"):

        opt = tf.keras.optimizers.Adam(learning_rate = 1e-4)
        loss = tf.keras.losses.CategoricalCrossentropy()
        model = Custom1DCNN(inp_shape=(_x_train.shape[1], _x_train.shape[2]), output_classes=y_train.shape[1])
        model.compile(optimizer = opt, loss=loss , metrics=['accuracy'])

        hist = model.fit(_x_train, y_train, batch_size=16, epochs = 70, verbose = False, validation_data = (_x_val,y_val), callbacks = cb_list)

        acc = hist.history['accuracy']
        val_acc = hist.history['val_accuracy']
        loss = hist.history['loss']
        val_loss = hist.history['val_loss']
        #eval
        vloss, vacc =  model.evaluate(_x_val, y_val, verbose = False)
        va.append(vacc)
        vl.append(vloss)

print("Mean L|ACC and STD L|ACC = {}|{} {}|{} ".format(
                                                        np.mean(np.array(vl)),
                                                        np.mean(np.array(va)),
                                                        np.std(np.array(vl)),
                                                        np.std(np.array(va))
                                                    )
    )
    


  0%|          | 0/10 [12:38<?, ?it/s]

Mean L|ACC and STD L|ACC = 1.103079915046692|0.5453703761100769 0.09630244746984863|0.09530292340334227 


### EXP IV
- Dataset:********* capilab_dataset2
- Classifier:****** Custom1DCNN
- Preprocessor:**** None
- Channel:********* SINGLE = (C4)


In [39]:
from tqdm import tqdm
vl = []
va = []

dataset_channel_map = {0: 'F4', 1: 'C4', 2: 'Pa', 3: 'Cz', 4: 'F3', 5: 'C3', 6: 'P3', 7: 'F7', 8: 'T3', 
                                    9: 'T5', 10: 'Fp1', 11: 'Fp2', 12: 'T4', 13: 'F8', 14: 'Fz', 15: 'Pz', 16: 'T6', 17: 'O2', 18: 'O1'}

target = [1]
_x_train = x_train[:,:,target].copy()
_x_test = x_test[:,:,target].copy()
_x_val = x_val[:,:,target].copy()


with tqdm(total = 10, position = 0, leave = True) as pbar:
    for e in tqdm(range(10), ncols = 100, position = 0, leave = True, desc ="getting the result>"):

        opt = tf.keras.optimizers.Adam(learning_rate = 1e-4)
        loss = tf.keras.losses.CategoricalCrossentropy()
        model = Custom1DCNN(inp_shape=(_x_train.shape[1], _x_train.shape[2]), output_classes=y_train.shape[1])
        model.compile(optimizer = opt, loss=loss , metrics=['accuracy'])

        hist = model.fit(_x_train, y_train, batch_size=16, epochs = 70, verbose = False, validation_data = (_x_val,y_val), callbacks = cb_list)

        acc = hist.history['accuracy']
        val_acc = hist.history['val_accuracy']
        loss = hist.history['loss']
        val_loss = hist.history['val_loss']
        #eval
        vloss, vacc =  model.evaluate(_x_val, y_val, verbose = False)
        va.append(vacc)
        vl.append(vloss)

print("Mean L|ACC and STD L|ACC = {}|{} {}|{} ".format(
                                                        np.mean(np.array(vl)),
                                                        np.mean(np.array(va)),
                                                        np.std(np.array(vl)),
                                                        np.std(np.array(va))
                                                    )
    )
    


  0%|          | 0/10 [12:36<?, ?it/s]

Mean L|ACC and STD L|ACC = 1.1097235202789306|0.5513888895511627 0.03194464242263654|0.02520491640029468 


| Channel Number        | Mean Validation Accuracy (std)    | Mean Validation Loss (std)    |
| ---                   | ---                               | ---                           |
| 1 ch (C4)             |  0.551388 (0.0252)                | 1.109723 (0.0319)             |
| 2 ch (C3, C4)         |  0.545370 (0.0953)                | 1.103079 (0.096)              |
| 10 ch remove blink    |  0.598533 (0.1125)                | 0.979109 (0.1387)             |
| 19 ch all             |  0.679861 (0.0197)                | 0.861941 (0.0381)             |

ValueError: Data cardinality is ambiguous:
  x sizes: 2160
  y sizes: 2400
Make sure all arrays contain the same number of samples.